In [1]:
import pandas as pd
from taxcrunch.multi_cruncher import Batch
import numpy as np
import bokeh
from bokeh.io import output_notebook, output_file, save
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, NumeralTickFormatter, Span, HoverTool, BoxAnnotation, Label, LabelSet, CustomJS
from bokeh.layouts import column, row, WidgetBox
from re import sub
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models.widgets import RadioButtonGroup, Slider, Dropdown, Div
from bokeh.palettes import Category20
from bokeh.core.properties import value

In [2]:
import bokeh
bokeh.__version__

'1.4.0'

In [3]:
import os
CURRENT_PATH = os.path.abspath(os.path.dirname('file'))

In [4]:
data_dict = {
 'year': [2026],
 'mstat': [1, 2],
 'page': [0],
 'sage': [0], 
 'depx': list(range(0,5)),
 'dep13': [0],
 'dep17': [0],
 'dep18': [0],
 'pwages': list(range(0,505000,5000)),
 'swages': [0],
 'dividends': [0],
 'intrec': [0],
 'stcg': [0],
 'ltcg': [0],
 'otherprop': [0],
 'nonprop': [0],
 'pensions': [0],
 'gssi': [0],
 'ui': [0],
 'proptax': [0],
 'otheritem': list(range(0,51000,1000)),
 'childcare': [0],
 'mortgage': list(range(0,51000,1000))
}

In [5]:
combos = [(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w)
     for a in data_dict['year']
     for b in data_dict['mstat'] 
     for c in data_dict['page'] 
     for d in data_dict['sage'] 
     for e in data_dict['depx'] 
     for f in data_dict['dep13']
     for g in data_dict['dep17']
     for h in data_dict['dep18'] 
     for i in data_dict['pwages'] 
     for j in data_dict['swages'] 
     for k in data_dict['dividends'] 
     for l in data_dict['intrec']
     for m in data_dict['stcg']
     for n in data_dict['ltcg'] 
     for o in data_dict['otherprop'] 
     for p in data_dict['nonprop'] 
     for q in data_dict['pensions'] 
     for r in data_dict['gssi']
     for s in data_dict['ui']
     for t in data_dict['proptax'] 
     for u in data_dict['otheritem'] 
     for v in data_dict['childcare'] 
     for w in data_dict['mortgage']]

In [6]:
df_combos = pd.DataFrame(combos)

In [7]:
df_combos.iloc[:,5] = df_combos.iloc[:,4]
df_combos.iloc[:,6] = df_combos.iloc[:,4]
df_combos.iloc[:,7] = df_combos.iloc[:,4]
df_combos.iloc[:,9] = np.where(df_combos.iloc[:,1] == 2, df_combos.iloc[:,8], 0)
df_combos.insert(0, 'RECID', df_combos.index+1)

In [8]:
df_combos.columns = range(df_combos.shape[1])

In [9]:
cols = ['ID', 'year', 'mstat', 'page', 'sage', 'depx', 'dep13', 'dep17', 'dep18', 'pwages', 'swages', 'dividends', 'intrec', 'stcg', 'ltcg', 'otherprop', 'nonprop', 'pensions', 'gssi', 'ui', 'proptax', 'otheritem', 'childcare', 'mortgage']
df_named = df_combos.copy()
df_named.columns = cols


In [10]:
b = Batch(df_combos)

In [11]:
baseline = b.create_table()
tcja_ext = b.create_table(reform_file=os.path.join(CURRENT_PATH, 'docs/WP_example/TCJA_ext.json'))

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0



In [12]:
baseline_merged = baseline.merge(df_named, on='ID')
keep = ['ID', 'Individual Income Tax', 'Wages', 'depx', 'mstat', 'otheritem', 'mortgage']
df_baseline_temp = baseline_merged[keep]
df_baseline = df_baseline_temp.rename(columns={'Individual Income Tax': 'itax base'})

df_merge_temp = df_baseline.merge(tcja_ext[['ID', 'Individual Income Tax']], on='ID')
df = df_merge_temp.rename(columns={'Individual Income Tax': 'itax ext'})

In [13]:
df_filter = df.loc[(df['depx'] == 1) & (df['mstat'] == 2) & (df['Wages'] == 50000) & (df['otheritem'] == 10000) & (df['mortgage'] == 0)]
keep = ['itax base', 'itax ext']
df_itax = df_filter[keep]
df_transposed = df_itax.transpose()
df_transposed.columns = ['itax']
x_vals = [0.5, 1.5]
df_transposed['x_vals'] = x_vals
src = ColumnDataSource(df_transposed)
src.data['itax']

array([ 325.296198, -518.909802])

In [14]:
df
# df.to_csv('tcja_ext_data.csv')

,ID,itax base,Wages,depx,mstat,otheritem,mortgage,itax ext
0,1.0,0.00000,0.0,0,1,0,0,0.0000
1,2.0,0.00000,0.0,0,1,0,1000,0.0000
2,3.0,0.00000,0.0,0,1,0,2000,0.0000
3,4.0,0.00000,0.0,0,1,0,3000,0.0000
4,5.0,0.00000,0.0,0,1,0,4000,0.0000
...,...,...,...,...,...,...,...,...
2627005,2627006.0,299378.17144,1000000.0,4,2,50000,46000,277582.8904
2627006,2627007.0,298982.17144,1000000.0,4,2,50000,47000,277212.8904
2627007,2627008.0,298586.17144,1000000.0,4,2,50000,48000,276842.8904
2627008,2627009.0,298190.17144,1000000.0,4,2,50000,49000,276472.8904


In [15]:
output_notebook()

Loading BokehJS ...

In [24]:
def modify_doc(doc):
    
    def find_val(mstat, deps, wages, salt, item):
        df_filter = df.loc[(df['depx'] == deps) & (df['mstat'] == mstat) & (df['Wages'] == wages) & (df['otheritem'] == salt) & (df['mortgage'] == item)]
        keep = ['itax base', 'itax ext']
        df_itax = df_filter[keep]
#         df_itax['dol_cut'] = (df_itax.loc[:,'itax base'] - df_itax.loc[:,'itax ext'])
        df_transposed = df_itax.transpose()
        df_transposed.columns = ['itax']
        x_vals = [0.5, 1.5]
        df_transposed['x_vals'] = x_vals
        itax_base = df_transposed.iloc[0,0]
        itax_ext = df_transposed.iloc[1,0]
        return ColumnDataSource(df_transposed)
    
    
    def make_plot(src):
        p = figure(x_range=["Current Law", "TCJA Extension"], plot_height=250, plot_width=400, toolbar_location=None, tools="")
        p.vbar(source=src, x='x_vals', top='itax', color='#ff7f0e', hover_fill_color='#ff7f0e',hover_fill_alpha=0.8, width=0.3, fill_alpha=0.5)
        hline = Span(location=0, dimension='width', line_color='black')
        p.renderers.extend([hline])
        p.xaxis.axis_line_color = '#d3d3d3'
        p.yaxis.axis_line_color = '#d3d3d3'
        p.yaxis[0].ticker.desired_num_ticks = 2
        p.xaxis.major_tick_line_color = None
        p.xaxis.minor_tick_line_color = None
        p.yaxis.minor_tick_line_color = None
        p.yaxis.major_tick_line_color = '#d3d3d3'
        p.yaxis[0].formatter = NumeralTickFormatter(format="$0,000")
        p.yaxis.axis_label = "Individual Income Tax"
#         p.yaxis.label_orientation = "horizontal"
        p.ygrid.grid_line_color = None
        p.xgrid.grid_line_color = None
        p.outline_line_color = None
        p.xaxis.axis_line_color = None
        p.title.text = 'Current Law vs TCJA Extension — 2026'        
        
        hover = HoverTool(tooltips='@itax{$0,000}')
        p.add_tools(hover)

        return p
    
    def update(attr, old, new):
        if mstat_button.labels[mstat_button.active] == 'Single':
            mstat = 1
        else:
            mstat = 2
            
        deps = deps_slider.value
        wages = wages_slider.value
        salt = salt_slider.value
        item = item_slider.value

        new_src = find_val(mstat=mstat, deps=deps, wages=wages, salt=salt, item=item)
        
        src.data.update(new_src.data)
    
    mstat_button = RadioButtonGroup(labels=["Single", "Married"], active=0)
    deps_slider = Slider(start=0, end=4, value=0, step=1, title="Number of Children")
    wages_slider = Slider(start=0, end=500000, value=0, step=10000, title="Wage Income", format="$0,000")
    salt_slider = Slider(start=0, end=50000, value=0, step=1000, title="State and Local Taxes Paid", format="$0,000")
    item_slider = Slider(start=0, end=50000, value=0, step=1000, title="Other Itemizable Expenses", format="$0,000")
    
    mstat_button.on_change('active', update)
    deps_slider.on_change('value', update)
    wages_slider.on_change('value', update)
    salt_slider.on_change('value', update)
    item_slider.on_change('value', update)

    controls = WidgetBox(mstat_button, deps_slider, wages_slider, salt_slider, item_slider)
    
    src = find_val(mstat=1, deps=0, wages=0, salt=0, item=0)
    
    p = make_plot(src)
    
    d = Div(text="In 2026...")
    d1 = Div(text=f"<i>Under <b>current law</b>, individual income tax liabilities are: <b>${src.data['itax'][0]}</b></i>")
    d2 = Div(text=f"<i>Under <b>TCJA extension</b>, individual income tax liabilities are: <b>${src.data['itax'][1]}</b></i>")
    d3 = Div(text=f"TCJA extension results in <b><font color='RebeccaPurple'>no tax change</font></b>")
    
    space = Div(text="  ")
    
    src.js_on_change('data', CustomJS(args=dict(d1=d1, d2=d2, d3=d3),
                                 code=js))
    
    layout = column(d, d1, d2, d3, space, row(controls, space, p))
    doc.add_root(layout)
    
handler = FunctionHandler(modify_doc)
app = Application(handler)

show(app, notebook_url='http://localhost:8889')


In [21]:
js = """
function commas(x) {
    return x.toString().replace(/\B(?=(\d{3})+(?!\d))/g, ",");
}
var v1 = Math.round(cb_obj.data['itax'][0]);
var v2 = Math.round(cb_obj.data['itax'][1]);

var cut = v1 - v2;
var increase = v2 - v1;

var cut_str = "tax cut of $";
var inc_str = "tax increase of $";
var no_change = "no tax change";

d1.text = `<i>Under <b>current law</b>, individual income tax liabilities are: <b>$${commas(v1)}</b></i>`;
d2.text = `<i>Under <b>TCJA extension</b>, individual income tax liabilities are: <b>$${commas(v2)}</b></i>`;

if (v1 > v2) {
    d3.text = `TCJA extension results in a <b>${cut_str.fontcolor("green")}${commas(cut).fontcolor("green")}</b>`;
} else if (v2 > v1) {
    d3.text = `TCJA extension results in a <b>${inc_str.fontcolor("red")}${commas(increase).fontcolor("red")}</b>`;
} else {
    d3.text = `TCJA extension results in <b>${no_change.fontcolor("RebeccaPurple")}</b>`;
}
"""